<a href="https://colab.research.google.com/github/sofiane1010/SIC_test/blob/main/nlp_test/problem_0801.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Quiz #0801

### "Text Classification with Keras"

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import os
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Embedding
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!wget https://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz

--2021-09-02 12:57:54--  https://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/x-gzip]
Saving to: ‘review_polarity.tar.gz.2’

review_polarity.tar 100%[===================>]   2.98M  4.77MB/s    in 0.6s    

2021-09-02 12:57:55 (4.77 MB/s) - ‘review_polarity.tar.gz.2’ saved [3127238/3127238]



In [ ]:
!tar xvzf review_polarity.tar.gz

txt_sentoken/neg/cv000_29416.txt
txt_sentoken/neg/cv001_19502.txt
txt_sentoken/neg/cv002_17424.txt
txt_sentoken/neg/cv003_12683.txt
txt_sentoken/neg/cv004_12641.txt
txt_sentoken/neg/cv005_29357.txt
txt_sentoken/neg/cv006_17022.txt
txt_sentoken/neg/cv007_4992.txt
txt_sentoken/neg/cv008_29326.txt
txt_sentoken/neg/cv009_29417.txt
txt_sentoken/neg/cv010_29063.txt
txt_sentoken/neg/cv011_13044.txt
txt_sentoken/neg/cv012_29411.txt
txt_sentoken/neg/cv013_10494.txt
txt_sentoken/neg/cv014_15600.txt
txt_sentoken/neg/cv015_29356.txt
txt_sentoken/neg/cv016_4348.txt
txt_sentoken/neg/cv017_23487.txt
txt_sentoken/neg/cv018_21672.txt
txt_sentoken/neg/cv019_16117.txt
txt_sentoken/neg/cv020_9234.txt
txt_sentoken/neg/cv021_17313.txt
txt_sentoken/neg/cv022_14227.txt
txt_sentoken/neg/cv023_13847.txt
txt_sentoken/neg/cv024_7033.txt
txt_sentoken/neg/cv025_29825.txt
txt_sentoken/neg/cv026_29229.txt
txt_sentoken/neg/cv027_26270.txt
txt_sentoken/neg/cv028_26964.txt
txt_sentoken/neg/cv029_19943.txt
txt_sentoken/n

#### Answer the following question by providing Python code:

1). Read in the movie review data from Cornell CS department. Carry out the EDA. <br>
- The data can be found [here](https://www.cs.cornell.edu/people/pabo/movie-review-data). <br>
- Download the “polarity dataset” and unzip. <br>
- Under the "txt_sentoken” folder, there are “pos” and “neg" subfolders. <br>

In [ ]:
reviews = load_files('txt_sentoken/')
my_docs, y = reviews.data, reviews.target

2). Carry out the data preprocessing: <br>
- Cleaning.
- Stopword removal.

In [ ]:
def cleantxt(txt):
    """
    Cleans the string passed. Cleaning Includes-
    1. remove special characters/symbols
    2. convert text to lower-case
    3. retain only alphabets
    4. remove words less than 3 characters
    5. remove stop-words
    """  
    # collecting english stop words from nltk-library
    stpw = stopwords.words('english')
    
    # Adding custom stop-words
    stpw.extend(['www','http','utc'])
    stpw = set(stpw)
    
    # using regex to clean the text
    txt = re.sub(r"\n", " ", txt)
    txt = re.sub("[\<\[].*?[\>\]]", " ", txt)
    txt = txt.lower()
    txt = re.sub(r"[^a-z ]", " ", txt)
    txt = re.sub(r"\b\w{1,3}\b", " ",txt)
    txt = " ".join([x for x in txt.split() if x not in stpw])
    return txt

corpus = []
for doc in my_docs:
  corpus.append(cleantxt(doc.decode('utf-8')))

corpus

['arnold schwarzenegger icon action enthusiasts since late lately films sloppy liners getting worse hard seeing arnold freeze batman robin especially says tons jokes million matter arnold signed another expensive blockbuster compare likes terminator series true lies even eraser called dark thriller devil gabriel byrne come upon earth impregnate woman robin tunney happens every years basically destroy world apparently chosen jericho cane arnold help trusty sidekick kevin pollack stop nothing devil take world parts actually absurd would right dogma film weak better blockbuster right sleepy hollow makes world enough look like star film anyway definitely seem like arnold movie type film sure gave chuckles well known liners seemed confused character film going understandable especially ending changed according sources aside form still walked much like past films sorry arnold maybe action days speaking action film hardly explosions fights devil made places explode arnold kicking devil butt e

3). Carry out label encoding by integers (required form by Keras):

In [ ]:
# Make a dictionary with the top words.
n_words = 2000  
words = []
for i in range(len(corpus)):
    words += nltk.word_tokenize(corpus[i])
top_words = pd.Series(words).value_counts().index
top_words = top_words[0:n_words]                     # Apply a limitation.
my_dict = {}
my_dict_inv = {}
for i in range(len(top_words)):
    my_dict_inv[i] = top_words[i]
    my_dict[top_words[i]] = i  

In [ ]:
# Convert the corpus into the label encoded form.
corpus_int =[]
for i in range(len(corpus)):
    words = nltk.word_tokenize(corpus[i])
    words2int = []
    for x in words:
        if x in my_dict:
            words2int += [my_dict[x]]
    corpus_int.append(words2int)

4). Prepare the data for AI: <br>
- Apply the padding.
- Split the data into training and testing.

In [ ]:
X = np.array(corpus_int)
y = np.array(y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
# Padding: newswire lengths are uniformly matched to maxlen.
X = sequence.pad_sequences(X, maxlen = 100)

# y is already binary. Thus, there is no need to covert to the one-hot-encoding scheme.

In [ ]:
#split the data into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.3, random_state = 1234, shuffle=True)

5). Define the AI model (Embedding + LSTM):

In [ ]:
n_neurons = 100                    # Neurons within each memory cell.
n_input = 500                     # Dimension of the embeding space. 

In [ ]:
my_model = Sequential()

my_model.add(Embedding(n_words, n_input, input_length=100))
my_model.add(LSTM(100))
my_model.add(Dense(1, activation='sigmoid'))

In [ ]:
my_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(my_model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 500)          1000000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               240400    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 1,240,501
Trainable params: 1,240,501
Non-trainable params: 0
_________________________________________________________________
None


6). Define the optimizer and compile the model:

In [ ]:
n_epochs = 15                      # Number of epochs.
batch_size = 100                    # Size of each batch.
learn_rate = 0.001

In [ ]:
my_model.fit(X_train,Y_train,batch_size=batch_size,epochs=n_epochs,verbose=1,callbacks=[EarlyStopping(monitor='accuracy', patience=3, min_delta=learn_rate)])

Epoch 1/15
14/14 [==============================] - 7s 414ms/step - loss: 0.6839 - accuracy: 0.6100
Epoch 2/15
14/14 [==============================] - 6s 406ms/step - loss: 0.4729 - accuracy: 0.8236
Epoch 3/15
14/14 [==============================] - 6s 407ms/step - loss: 0.2312 - accuracy: 0.9143
Epoch 4/15
14/14 [==============================] - 6s 407ms/step - loss: 0.1060 - accuracy: 0.9671
Epoch 5/15
14/14 [==============================] - 6s 401ms/step - loss: 0.0581 - accuracy: 0.9843
Epoch 6/15
14/14 [==============================] - 6s 407ms/step - loss: 0.0458 - accuracy: 0.9850
Epoch 7/15
14/14 [==============================] - 6s 402ms/step - loss: 0.0269 - accuracy: 0.9929
Epoch 8/15
14/14 [==============================] - 6s 403ms/step - loss: 0.0221 - accuracy: 0.9943
Epoch 9/15
14/14 [==============================] - 6s 402ms/step - loss: 0.0144 - accuracy: 0.9957
Epoch 10/15
14/14 [==============================] - 6s 397ms/step - loss: 0.0138 - accuracy: 0.9971

In [ ]:
my_model.evaluate(X_test,Y_test)

19/19 [==============================] - 2s 69ms/step - loss: 0.9710 - accuracy: 0.7817


[0.9709529876708984, 0.7816666960716248]